In [14]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

In [4]:
base=pd.read_csv('PETR4.SA.csv')
base=base.dropna()
base_treinamento=base.iloc[:, 1:2].values

In [8]:
normalizador=MinMaxScaler(feature_range=(0,1))
base_treinamento_normalizada = normalizador.fit_transform(base_treinamento)

In [9]:
previsores=[]
preco_real=[]
for i in range(90,850):
    previsores.append(base_treinamento_normalizada[i-90:i,0])
    preco_real.append(base_treinamento_normalizada[i,0])

In [10]:
previsores,preco_real=np.array(previsores), np.array(preco_real)

In [13]:
previsores = np.reshape(previsores, (previsores.shape[0], previsores.shape[1], 1))

In [15]:
regressor=Sequential()
regressor.add(LSTM(units=100,return_sequences=True, input_shape= (previsores.shape[1],1)))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units=50,return_sequences=True))
regressor.add(Dropout(0.3))

regressor.add(Dense(units=1,activation='linear'))

regressor.compile(optimizer='rmsprop',loss='mean_squared_error',metrics=['mean_absolute_error'])

c:\Users\samue\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
results=regressor.predict(base_treinamento)